$S = QK^T \in \mathbb{R}^{N\times N}$,   
$P = sfotmax(S)\in \mathbb{R}^{N\times N}$,        
$O =PV \in \mathbb{R}^{N\times d} $      

In [51]:
import torch
torch.set_default_dtype(torch.float64)
seq_len = 12
dim = 5 # head_dim
Q = torch.randn((seq_len, dim))
K = torch.randn((seq_len, dim))
V = torch.randn((seq_len, dim))
S = torch.matmul(Q, K.transpose(0, 1))
P = torch.softmax(S, dim=-1)
O_ori = torch.matmul(P, V)

前向过程    
外层 切片q，o（保证引用），初始化m_i l_i     
    内层切边k，v    
        计算s    
        计算m_ij (直接m_j与m_i取最大,m_j是当前块的)    
        计算p_hat （e^(x-max)）    
        更新计算 l_i     
        更新计算 o    
        更新mi    

    记录m_i +ln^l_i     

In [ ]:
block_qo= 2
block_kv = 3
O = torch.zeros_like(Q)
M = torch.zeros(seq_len) # 保存max + ln^l
for block_idx in range(seq_len // block_qo): # 按块遍历 Q
    q = Q[block_idx*block_qo:(block_idx+1)*block_qo, :]
    o = O[block_idx*block_qo:(block_idx+1)*block_qo, :]
    m_i= torch.ones(block_qo)*float('-inf')
    l_i = torch.zeros(block_qo)
    for block_kv_idx in range(seq_len // block_kv): # 按块遍历 K, V
        k = K[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv, :]
        v = V[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv, :]

        # 求局部值
        s =torch.matmul(q, k.transpose(0, 1))
        m_ij = torch.maximum(torch.max(s,dim=-1)[0],m_i) 
        # torch.max 返回最大值[0]和最大值的索引[1]  ,求出包含当前处理bolck_j的局部最大值
        P_hat = torch.exp(s - m_ij[:,None]) # exp(s - max(s))
        l_ij = torch.sum(P_hat, dim=-1)
        alpha = torch.exp(m_i - m_ij)

        # 更新
        o[...] = o * alpha[:,None]
        o[...] += torch.matmul(P_hat, v)
        l_i = alpha*l_i + l_ij # 更新
        m_i = m_ij  #更新


    o[:] /= l_i[:,None]
    M[block_idx*block_qo:(block_idx+1)*block_qo] = m_i + torch.log(l_i)


torch.allclose(O,O_ori),torch.max(O-O_ori),M

(True,
 tensor(6.6613e-16),
 tensor([2.4857, 2.9453, 3.6392, 4.5479, 2.4173, 8.1766, 2.5292, 3.5198, 2.6965,
         3.8635, 2.6707, 4.4407]))

tensor([[-12.8096,  12.9031,  15.9182,  15.9139],
        [ -3.2148,  -4.7085, -33.2408,  11.8141],
        [ -7.8927,  -3.8054,   4.1934,   2.7760],
        [-10.2862,  -3.0962,  21.6827,  10.1150]])